In [1]:
# https://maartengr.github.io/BERTopic/getting_started/zeroshot/zeroshot.html#example

import re
import os

import joblib
import numpy as np
import openai
import pandas as pd
from bertopic import BERTopic
from bertopic.backend import OpenAIBackend
from bertopic.representation import (KeyBERTInspired, OpenAI,
                                     ZeroShotClassification)
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

2024-10-09 07:14:40.525227: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-09 07:14:40.543770: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-09 07:14:40.549321: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-09 07:14:40.563307: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-09 07:14:41.591555: W tensorflow/compiler/tf2

In [2]:
df = pd.read_csv('/media/nas-elias/drive/UFRN/Disciplinas/2024-2/imd1107-nlp/data/flexitarianism.csv')
df

,id,motivo_de_se_considerar_flex,fonteinfo_flex,motivacao,dificuldade,genero,cor_raca,escolaridade
0,1,nao me considero,amigos,aversao,condenacao moral,mulher_cis,branca,superior
1,2,reducao do impacto_ambiental,escola_universidade,impacto_ambiental,habito cultural adquirido,homem_cis,branca,doutorado
2,3,porque a carne bovina de certa forma me faz ma...,amigos,saude,preco e variedade de outras proteinas vegetais...,homem_cis,branca,mestrado
3,4,financeiro,amigos,religiao,consumo desde crianca,pnr,parda,superior
4,5,pois limitei o consumo para nao desenvolver ac...,midias,saude,nenhuma,mulher_cis,branca,superior
...,...,...,...,...,...,...,...,...
1024,1025,necessidade financeira,midias,saude,nenhuma,mulher_cis,preta,medio
1025,1026,o amor pelos animais,escola_universidade,etica_animal,habito alimentae,mulher_cis,branca,superior
1026,1027,porque eu deliberadamente tenho reduzido meu c...,midias,saude,as pessoas fora da minha casa nao estarem envo...,homem_cis,branca,medio
1027,1028,"porque cuido de mim, do meu corpo e do meu pla...",escola_universidade,impacto_ambiental,nenhuma,mulher_cis,branca,superior


In [3]:
df_reason = df.copy()
df_challenge = df.copy()

In [4]:
# Rename the column 'motivo_de_se_considerar_flex' to 'input_text' in the df_reason DataFrame
# This standardizes the column name for easier processing
df_reason.rename(columns={'motivo_de_se_considerar_flex': 'input_text'}, inplace=True)

# Rename the column 'dificuldade' to 'input_text' in the df_challenge DataFrame
# This standardizes the column name for easier processing
df_challenge.rename(columns={'dificuldade': 'input_text'}, inplace=True)

# Calculate the length of the text in the 'input_text' column for each row in df_reason
# Store the lengths in a new column called 'len_text'
df_reason['len_text'] = df_reason.input_text.str.len()

# Calculate the length of the text in the 'input_text' column for each row in df_challenge
# Store the lengths in a new column called 'len_text'
df_challenge['len_text'] = df_challenge.input_text.str.len()

# Calculate the 25th percentile (first quartile) of the text lengths in df_reason
# This value will be used to filter out shorter texts
reason_quartile = df_reason.len_text.quantile(0.25)

# Calculate the 25th percentile (first quartile) of the text lengths in df_challenge
# This value will be used to filter out shorter texts
challenge_quartile = df_challenge.len_text.quantile(0.25)

# Filter df_reason to include only rows where the text length is greater than or equal to the first quartile
# This removes the shortest 25% of texts
df_reason = df_reason.query('len_text >= @reason_quartile')

# Filter df_challenge to include only rows where the text length is greater than or equal to the first quartile
# This removes the shortest 25% of texts
df_challenge = df_challenge.query('len_text >= @challenge_quartile')

# Drop rows with missing values in the 'input_text' column from df_reason
# This ensures that all remaining rows have valid text data
df_reason.dropna(inplace=True, subset=['input_text'])

# Drop rows with missing values in the 'input_text' column from df_challenge
# This ensures that all remaining rows have valid text data
df_challenge.dropna(inplace=True, subset=['input_text'])

# Display the shapes of the filtered DataFrames to see the number of remaining rows and columns
df_reason.shape, df_challenge.shape

((778, 9), (779, 9))

## Cleaning our data and saving the transformed data

In [5]:
import re
from sklearn.pipeline import Pipeline # Pipeline applies a list of transforms. You can also add an estimator at the end, so it will be completely encapsulated.
from sklearn.preprocessing import FunctionTransformer # FunctionTransformer allows to apply an arbitrary function to the data, so we can use it in the pipeline
import unicodedata
import spacy
from typing import List

def remove_excessive_spaces(text: str) -> str:
    """
    This function removes excessive spaces from the text.

    Args:
        text (str): The input text.

    Returns:
        str: The text with excessive spaces removed.
    """
    return re.sub(r'\s+', ' ', text).strip() 

def remove_repeated_non_word_characters(text: str) -> str:
    """
    This function removes repeated non-word characters from the text.

    Args:
        text (str): The input text.

    Returns:
        str: The text with repeated non-word characters removed.
    """
    return re.sub(r'(\W)\1+', r'\1', text).strip()

def remove_first_line_from_text(text: str) -> str:
    """
    This function removes the first line from the text.

    Args:
        text (str): The input text.

    Returns:
        str: The text with the first line removed.
    """
    return re.sub(r'^.*\n', '', text).strip()

def remove_last_line_from_text(text: str) -> str:
    """
    This function removes the last line from the text.

    Args:
        text (str): The input text.

    Returns:
        str: The text with the last line removed.
    """
    return re.sub(r'\n.*$', '', text).strip()

def fix_isolated_commas_in_text(text: str) -> str:
    """
    This function fixes isolated commas in the text.

    Args:
        text (str): The input text.

    Returns:
        str: The text with isolated commas fixed.
    """
    text = re.sub(r' ([.,:;!?])', r'\1', text)
    return text.strip()

def keep_words_longer_than(text: str, min_length: int = 2) -> str:
    """
    This function keeps only the words in the text that are longer than a given length.

    Args:
        text (str): The input text.
        min_length (int, optional): The minimum length of the words to keep. Defaults to 2.

    Returns:
        str: The text with only the words longer than the given length.
    """
    return ' '.join([word for word in text.split() if len(word) > min_length])

def keep_only_alphabet_characters(text: str) -> str:
    """
    This function keeps only the alphabet characters in the text.

    Args:
        text (str): The input text.

    Returns:
        str: The text with only the alphabet characters.
    """
    return re.sub(r'[^a-zA-Z]', ' ', text).strip()

def remove_accents_from_text(text: str) -> str:
    """
    This function removes accents from the text.

    Args:
        text (str): The input text.

    Returns:
        str: The text with accents removed.
    """
    return unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')

def lemmatize_text_with_spacy(text: str) -> str:
    """
    This function lemmatizes the text using the Spacy library.

    Args:
        text (str): The input text.

    Returns:
        str: The lemmatized text.
    """
    doc = nlp_spacy(text)
    return ' '.join([token.lemma_ for token in doc])


pipeline_clean_text = Pipeline([
    ('remove_first_line_from_text', FunctionTransformer(remove_first_line_from_text)),
    ('remove_last_line_from_text', FunctionTransformer(remove_last_line_from_text)),
    ('remove_excessive_spaces', FunctionTransformer(remove_excessive_spaces)),
    ('remove_repeated_non_word_characters', FunctionTransformer(remove_repeated_non_word_characters)),
    ('fix_isolated_commas_in_text', FunctionTransformer(fix_isolated_commas_in_text)),
])


nlp_spacy = spacy.load('pt_core_news_sm')



In [6]:
# We can apply the pipeline to the data
df_reason['input_text_clean'] = df_reason['input_text'].apply(pipeline_clean_text.transform)

df_reason['input_text_clean_simplified'] = df_reason['input_text_clean'].apply(lemmatize_text_with_spacy)
df_reason['input_text_clean_simplified'] = df_reason['input_text_clean_simplified'].apply(remove_accents_from_text)
df_reason['input_text_clean_simplified'] = df_reason['input_text_clean_simplified'].apply(keep_words_longer_than)
df_reason['input_text_clean_simplified'] = df_reason['input_text_clean_simplified'].str.lower()
df_reason['input_text_clean_simplified'] = df_reason['input_text_clean_simplified'].apply(keep_only_alphabet_characters)
df_reason['input_text_clean_simplified'] = df_reason['input_text_clean_simplified'].apply(remove_excessive_spaces)


In [7]:
# We can apply the pipeline to the data
df_challenge['input_text_clean'] = df_challenge['input_text'].apply(pipeline_clean_text.transform)

df_challenge['input_text_clean_simplified'] = df_challenge['input_text_clean'].apply(lemmatize_text_with_spacy)
df_challenge['input_text_clean_simplified'] = df_challenge['input_text_clean_simplified'].apply(remove_accents_from_text)
df_challenge['input_text_clean_simplified'] = df_challenge['input_text_clean_simplified'].apply(keep_words_longer_than)
df_challenge['input_text_clean_simplified'] = df_challenge['input_text_clean_simplified'].str.lower()
df_challenge['input_text_clean_simplified'] = df_challenge['input_text_clean_simplified'].apply(keep_only_alphabet_characters)
df_challenge['input_text_clean_simplified'] = df_challenge['input_text_clean_simplified'].apply(remove_excessive_spaces)

In [8]:
# Print the shape of the df_reason DataFrame before dropping rows with missing values
# This shows the number of rows and columns before the cleaning process
print(df_reason.shape)

# Drop rows from df_reason where the 'input_text_clean' column has missing values
# This ensures that all remaining rows have valid cleaned text data
df_reason.dropna(inplace=True, subset=['input_text_clean'])

# Print the shape of the df_reason DataFrame after dropping rows with missing values
# This shows the number of rows and columns after the cleaning process, allowing comparison with the previous shape
print(df_reason.shape)

(778, 11)
(778, 11)


In [9]:
# Print the shape of the df_challenge DataFrame before dropping rows with missing values
# This shows the number of rows and columns before the cleaning process
print(df_challenge.shape)

# Drop rows from df_challenge where the 'input_text_clean' column has missing values
# This ensures that all remaining rows have valid cleaned text data
df_challenge.dropna(inplace=True, subset=['input_text_clean'])

# Print the shape of the df_challenge DataFrame after dropping rows with missing values
# This shows the number of rows and columns after the cleaning process, allowing comparison with the previous shape
print(df_challenge.shape)

(779, 11)
(779, 11)


In [10]:
# Import the spaCy library for advanced natural language processing tasks
import spacy

# Import the stopwords module from the nltk.corpus package
# This module provides a list of common stop words for various languages
from nltk.corpus import stopwords

# Get the list of Portuguese stop words from the NLTK library
# These are common words that are usually filtered out in text processing
stopwords_nltk = stopwords.words('portuguese')

# Load the small Portuguese language model from spaCy
# This model includes pre-trained word vectors, part-of-speech tags, named entity recognition, and more
nlp = spacy.load('pt_core_news_sm')

# Access the set of default stop words for the Portuguese language from the loaded spaCy model
stopwords_spacy = nlp.Defaults.stop_words

# Combine the stop words from both NLTK and spaCy into a single set
# Using a set ensures that each stop word appears only once, even if it is present in both lists
both_stopwords = set(stopwords_nltk) | set(stopwords_spacy)

# Calculate and display the total number of unique stop words in the combined set
len(both_stopwords)

500

In [11]:
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import MaximalMarginalRelevance
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
import re

In [12]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv

# Load environment variables from a .env file. This is necessary to access the OPENAI_API_KEY environment variable.
load_dotenv()

openai_client = ChatOpenAI(model='gpt-4o-mini')
max_tokens = 8191  # the maximum for text-embedding-3-large is 8191


In [13]:
# Check if any text in the 'input_text_clean' column of df_reason is longer than max_tokens
# This is done to ensure that the texts do not exceed the token limit, which might be required for certain NLP models or APIs
# openai_client.get_num_tokens(text) returns the number of tokens in the given text
long_texts_reason = [
    text for text in df_reason['input_text_clean'] 
    if openai_client.get_num_tokens(text) > max_tokens
]

# Check if any text in the 'input_text_clean' column of df_challenge is longer than max_tokens
# This is done to ensure that the texts do not exceed the token limit, which might be required for certain NLP models or APIs
# openai_client.get_num_tokens(text) returns the number of tokens in the given text
long_texts_challenge = [
    text for text in df_challenge['input_text_clean'] 
    if openai_client.get_num_tokens(text) > max_tokens
]

# Print the number of texts longer than max_tokens for both reasons and challenges
# This provides a quick summary of how many texts exceed the token limit in each dataset
print(f'Number of texts longer than {max_tokens} tokens: {len(long_texts_reason)} (reasons) and {len(long_texts_challenge)} (challenges)')

Number of texts longer than 8191 tokens: 0 (reasons) and 0 (challenges)


In [14]:
# According to OpenAI pricing website, the cost of the embedding API is $0.13 per million tokens
# https://openai.com/pricing/

from typing import List

def calculate_total_cost(text_list: List[str], cost_per_million_tokens: float) -> float:
    """
    This function calculates the total cost of processing a list of texts, 
    based on a cost per million tokens.

    Args:
        text_list (List[str]): A list of texts to be processed.
        cost_per_million_tokens (float): The cost of processing a million tokens.

    Returns:
        float: The total cost of processing the texts.
    """
    # Initialize the total number of tokens
    total_token_count = 0

    # Iterate over each text in the list
    for text in text_list:
        # Use the OpenAI client to get the number of tokens in the text
        # and add it to the total token count
        total_token_count += openai_client.get_num_tokens(text)

    # Calculate the total cost by multiplying the total token count by the cost per million tokens
    # and dividing by 1,000,000 (since the cost is per million tokens)
    total_cost = total_token_count * cost_per_million_tokens / 1_000_000

    return total_cost

# Concatenate the 'input_text_clean' columns from df_reason and df_challenge into a single list
# This combines the cleaned text data from both DataFrames for cost calculation
all_texts_concat = df_reason['input_text_clean'].values.tolist() + df_challenge['input_text_clean'].values.tolist()

# Calculate and print the total cost of using the embedding API for the concatenated texts
# The cost is calculated using the calculate_total_cost function with a rate of $0.13 per million tokens
print(f'Cost of embedding API for {len(all_texts_concat)} texts: ${calculate_total_cost(all_texts_concat, 0.13):.4f} USD')

Cost of embedding API for 1557 texts: $0.0043 USD


In [15]:
df_reason

,id,input_text,fonteinfo_flex,motivacao,dificuldade,genero,cor_raca,escolaridade,len_text,input_text_clean,input_text_clean_simplified
2,3,porque a carne bovina de certa forma me faz ma...,amigos,saude,preco e variedade de outras proteinas vegetais...,homem_cis,branca,mestrado,123,porque a carne bovina de certa forma me faz ma...,porque carne bovino certo forma fazer mal deix...
4,5,pois limitei o consumo para nao desenvolver ac...,midias,saude,nenhuma,mulher_cis,branca,superior,138,pois limitei o consumo para nao desenvolver ac...,pois limitar consumo para nao desenvolver acid...
6,7,so como carne quando nao ha alternativa nao ti...,amigos,impacto_ambiental,gostar do sabor,mulher_cis,branca,mestrado,297,so como carne quando nao ha alternativa nao ti...,como carne quando nao alternativo nao ter pens...
7,8,ja uns anos desde a adolescencia (nos anos 200...,midias,aversao,"na adolescencia e de novo na vida adulta, eu o...",mulher_cis,branca,mestrado,426,ja uns anos desde a adolescencia (nos anos 200...,ano desde adolescencia ano optar por reduzir c...
8,9,porque ao longo do tempo entendi que a reducao...,amigos,impacto_ambiental,nenhuma,mulher_cis,preta,superior,153,porque ao longo do tempo entendi que a reducao...,porque longo tempo entendi que reducao carne n...
...,...,...,...,...,...,...,...,...,...,...,...
1022,1023,consumo carne apenas na refeicao do almoco,escola_universidade,impacto_ambiental,substitulo que se iguale ao sabor,mulher_cis,branca,medio,42,consumo carne apenas na refeicao do almoco,consumo carne apenas refeicao almoco
1023,1024,porque nao consumo carne em todas as refeicões,outros,saude,gostar do sabor e nao poder retirar a carne po...,mulher_cis,branca,doutorado,46,porque nao consumo carne em todas as refeicões,porque nao consumo carne todo refeicao
1026,1027,porque eu deliberadamente tenho reduzido meu c...,midias,saude,as pessoas fora da minha casa nao estarem envo...,homem_cis,branca,medio,61,porque eu deliberadamente tenho reduzido meu c...,porque deliberadamente ter reduzir meu consumo...
1027,1028,"porque cuido de mim, do meu corpo e do meu pla...",escola_universidade,impacto_ambiental,nenhuma,mulher_cis,branca,superior,50,"porque cuido de mim, do meu corpo e do meu pla...",porque cuer meu corpo meu planeta


In [22]:
# Initialize the OpenAI embeddings model with the specified parameters
# - model: The name of the model to use for generating embeddings ('text-embedding-3-large')
# - dimensions: The dimensionality of the embeddings (3072)
# This model will convert text into high-dimensional vectors that capture semantic meaning
openai_embeddings = OpenAIEmbeddings(model='text-embedding-3-large', dimensions=3072)

# Generate embeddings for the 'input_text_clean' column of df_reason
# embed_documents takes a list of texts and returns their embeddings
# These embeddings can be used for various downstream tasks such as clustering, classification, or similarity search
embeddings_reason = openai_embeddings.embed_documents(df_reason['input_text_clean'].values.tolist())

# Generate embeddings for the 'input_text_clean' column of df_challenge
# embed_documents takes a list of texts and returns their embeddings
# These embeddings can be used for various downstream tasks such as clustering, classification, or similarity search
embeddings_challenge = openai_embeddings.embed_documents(df_challenge['input_text_clean'].values.tolist())

In [23]:
# Convert the list of embeddings for df_reason into a pandas Series and then to a DataFrame
# This creates a DataFrame where each row corresponds to an embedding
embeddings_df_reason = pd.Series(embeddings_reason).to_frame()

# Convert the list of embeddings for df_challenge into a pandas Series and then to a DataFrame
# This creates a DataFrame where each row corresponds to an embedding
embeddings_df_challenge = pd.Series(embeddings_challenge).to_frame()

# Rename the column of the embeddings DataFrame for df_reason to 'openai_embedding'
# This makes it clear that the column contains embeddings generated by the OpenAI model
embeddings_df_reason.columns = ['openai_embedding']

# Rename the column of the embeddings DataFrame for df_challenge to 'openai_embedding'
# This makes it clear that the column contains embeddings generated by the OpenAI model
embeddings_df_challenge.columns = ['openai_embedding']

# Display the embeddings DataFrame for df_reason
# This allows you to inspect the DataFrame and verify that the embeddings have been correctly converted and stored
embeddings_df_reason

,openai_embedding
0,"[0.006285921670496464, 0.014335720799863338, -..."
1,"[0.009271223098039627, 0.028493599966168404, -..."
2,"[0.006935932207852602, 0.025884093716740608, -..."
3,"[0.01735425367951393, 0.008659853599965572, -0..."
4,"[0.029077960178256035, 0.009871339425444603, -..."
...,...
586,"[-0.013749081641435623, -0.007834980264306068,..."
587,"[0.0032380360644310713, 0.02231498621404171, -..."
588,"[-0.0056138550862669945, 0.045160576701164246,..."
589,"[0.028120173141360283, -0.004942643456161022, ..."


In [24]:
# Calculate the length of the embedding vector for the first row in the embeddings_df_reason DataFrame
# - iloc[0] selects the first row of the DataFrame
# - ['openai_embedding'] accesses the 'openai_embedding' column, which contains the embedding vector
# - len() calculates the length of the embedding vector
# This is useful to verify the dimensionality of the embeddings generated by the OpenAI model
embedding_length = len(embeddings_df_reason.iloc[0]['openai_embedding'])

# Display the length of the embedding vector
# This confirms that the embeddings have the expected dimensionality (e.g., 3072 dimensions)
embedding_length

3072

In [25]:
# Concatenate the embeddings DataFrame with the original df_reason DataFrame
# - pd.concat() is used to concatenate DataFrames along a specified axis
# - axis=1 specifies concatenation along columns, adding the 'openai_embedding' column to df_reason
# This adds the embeddings as a new column to the original DataFrame
df_reason = pd.concat([df_reason, embeddings_df_reason], axis=1)

# Do the same for the df_challenge DataFrame
df_challenge = pd.concat([df_challenge, embeddings_df_challenge], axis=1)

# Display the first few rows of the updated df_reason DataFrame
# This allows you to verify that the embeddings have been correctly added as a new column
df_reason.head()

,id,input_text,fonteinfo_flex,motivacao,dificuldade,genero,cor_raca,escolaridade,len_text,input_text_clean,input_text_clean_simplified,openai_embedding
2,3.0,porque a carne bovina de certa forma me faz ma...,amigos,saude,preco e variedade de outras proteinas vegetais...,homem_cis,branca,mestrado,123.0,porque a carne bovina de certa forma me faz ma...,porque carne bovino certo forma fazer mal deix...,"[0.006935932207852602, 0.025884093716740608, -..."
4,5.0,pois limitei o consumo para nao desenvolver ac...,midias,saude,nenhuma,mulher_cis,branca,superior,138.0,pois limitei o consumo para nao desenvolver ac...,pois limitar consumo para nao desenvolver acid...,"[0.029077960178256035, 0.009871339425444603, -..."
6,7.0,so como carne quando nao ha alternativa nao ti...,amigos,impacto_ambiental,gostar do sabor,mulher_cis,branca,mestrado,297.0,so como carne quando nao ha alternativa nao ti...,como carne quando nao alternativo nao ter pens...,"[-0.020992489531636238, 0.03462756425142288, -..."
7,8.0,ja uns anos desde a adolescencia (nos anos 200...,midias,aversao,"na adolescencia e de novo na vida adulta, eu o...",mulher_cis,branca,mestrado,426.0,ja uns anos desde a adolescencia (nos anos 200...,ano desde adolescencia ano optar por reduzir c...,"[-0.02649660035967827, 0.007609663996845484, 0..."
8,9.0,porque ao longo do tempo entendi que a reducao...,amigos,impacto_ambiental,nenhuma,mulher_cis,preta,superior,153.0,porque ao longo do tempo entendi que a reducao...,porque longo tempo entendi que reducao carne n...,"[0.017545772716403008, 0.011176908388733864, -..."


In [26]:
df_challenge.head()

,id,motivo_de_se_considerar_flex,fonteinfo_flex,motivacao,input_text,genero,cor_raca,escolaridade,len_text,input_text_clean,input_text_clean_simplified,openai_embedding
2,3.0,porque a carne bovina de certa forma me faz ma...,amigos,saude,preco e variedade de outras proteinas vegetais...,homem_cis,branca,mestrado,58.0,preco e variedade de outras proteinas vegetais...,precar variedade outro proteina vegetal dispon...,"[-0.006870491895824671, 0.019664281979203224, ..."
5,6.0,porque tento nao comer carne todo dia,amigos,impacto_ambiental,desconfortos abdominais quando o consumo de fi...,homem_cis,branca,doutorado,59.0,desconfortos abdominais quando o consumo de fi...,desconforto abdominal quando consumo fibra grande,"[-0.017436495050787926, -0.009107337333261967,..."
7,8.0,ja uns anos desde a adolescencia (nos anos 200...,midias,aversao,"na adolescencia e de novo na vida adulta, eu o...",mulher_cis,branca,mestrado,590.0,eu acho que eu vivo em uma constante balanca d...,achar que viver constante balanca desequilibrar,"[0.005193557124584913, 0.034651461988687515, -..."
9,10.0,por causa da industria de alimentos do aquecim...,midias,impacto_ambiental,alimentos que possam substituir na dieta por u...,homem_cis,branca,mestrado,62.0,alimentos que possam substituir na dieta por u...,alimento que poder substituir dieta por preco ...,"[-0.005540707614272833, 0.04621240869164467, 0..."
10,11.0,na verdade venho tentando ser vegana ha anos c...,outros,impacto_ambiental,capacidade financeira. acompanhamento nutricio...,mulher_cis,branca,superior,248.0,capacidade financeira. acompanhamento nutricio...,capacidade financeiro acompanhamento nutricion...,"[-0.013430182822048664, 0.019844232127070427, ..."


In [38]:
df_reason.dropna(subset=['input_text'], inplace=True)

In [39]:
texts = df_reason.input_text.values.tolist()
texts[:5]

['porque a carne bovina de certa forma me faz mal deixa o intestino "pesado" e prefiro nao consumir caso tenham outras opcões',
 'pois limitei o consumo para nao desenvolver acido úrico e por ver muitas reportagens sobre os maleficios do alto consumo de carne vermelha',
 'so como carne quando nao ha alternativa nao tinha pensado ou conhecia essa terminologia sempre me denominei carnivora gosto de carne mas minha filha virou vegetariana entao eu acabei me adaptando e so como carne quando da vontade tipo em marco ate agora comi umas 5 vezes e hj ja e dia 24 de marco',
 'ja uns anos desde a adolescencia (nos anos 2000) eu optei por reduzir o consumo de carne vermelha por nao gostar do sabor e de como eu fico depois que consumo o que nao ocorre tanto quando eu como frango porco (um pouco por causa da gordura) e peixe exceto da forma como as comidas sao temperadas ou cozidas so que tambem nao e algo que me faca falta e mesmo assim de vez em quando fico umas semanas sem consumir carne nenhum

In [49]:
# Initialize embeddings model
embedding_model_langchain = OpenAIEmbeddings(model="text-embedding-3-large")


In [50]:
embeddings = embedding_model_langchain.embed_documents(texts)

In [51]:
embeddings = np.array(embeddings)
embeddings.shape

(591, 1536)

In [52]:

prompt = """
Eu tenho um tópico que contém os seguintes documentos:
[DOCUMENTS]
O tópico é descrito pelas seguintes palavras-chave: [KEYWORDS]

Com base nas informações acima, extraia um rótulo de tópico curto no seguinte formato:
topic: <topic label>

O rótulo do tópico deve ser uma descrição concisa do conteúdo dos documentos e palavras-chave fornecidos, não fazendo referência a informações específicas de anos, datas, números ou nomes de pessoas. O rótulo do tópico deve ter até 3 palavras.
"""

# Initialize OpenAI client
openai_client = openai.Client()


# Initialize embedding model for BERTopic
embedding_model_bertopic = OpenAIBackend(client=openai_client, embedding_model="text-embedding-3-small") # Same as embedding_model_langchain, that was used to pre-compute the embeddings of our history

# Initialize BERTopic model with specified parameters
representation_model = OpenAI(openai_client, model="gpt-4o-2024-08-06", chat=True, prompt=prompt) # This class comes from the bertopic.representation module

In [109]:
topic_model = BERTopic(
    embedding_model=embedding_model_bertopic,
    min_topic_size=15,
    zeroshot_topic_list=["exclui alguns animais", "não come carne vermelha", "diminui a frequência de consumo", "não considera carne essencial"],
    zeroshot_min_similarity=0.8,
    nr_topics=8,
    representation_model=KeyBERTInspired()#representation_model # this will make len(topics) requests to the OpenAI API
) # KeyBERTInspired

topics, _ = topic_model.fit_transform(texts, embeddings=embeddings)

In [110]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,99,-1_carne_alimentacao_carnes_consumo,"[carne, alimentacao, carnes, consumo, animais,...",[porque ha dias na semana em que nao como carn...
1,0,90,0_alimentacao_preocupacao_consumo_ambientais,"[alimentacao, preocupacao, consumo, ambientais...",[me preocupo com as acões do agronegocio e seu...
2,1,402,1_vegetariana_comendo_alimentacao_vegana,"[vegetariana, comendo, alimentacao, vegana, re...",[na verdade venho tentando ser vegana ha anos ...


In [111]:
topic_model.visualize_documents(texts, embeddings=embeddings)

In [112]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Optional, List, Literal, Optional
from langchain_openai import ChatOpenAI
model_openai = ChatOpenAI(
    model='gpt-4o-mini', 
    temperature=0
) 

In [131]:
class ClassificaFlexitarianismo(BaseModel):
    """Classifica uma entrada textual quanto às razões que fazem uma pessoa se considerar flexitariana"""

    reason: Literal["exclui alguns animais", "diminui a frequência de consumo", "não considera carne essencial", "preocupação com a saúde ou ambiente", "outros"] = Field(default_factory=str, description="Razão que faz uma pessoa se considerar flexitariana")

zsl_classifier = model_openai.with_structured_output(ClassificaFlexitarianismo).with_retry(
    wait_exponential_jitter=True,
    stop_after_attempt=5
)

# class HaRazaoParaFlexitarianismo(BaseModel):
#     """
#     Classifica se há, no texto de entrada, razões para aderir a uma dieta flexitariana.
#     """

#     has_reason: bool = Field(default=False, description="Indica se, EXPLICITAMENTE, há razões para aderir a uma dieta flexitariana no texto de entrada.'")

# zsl_classifier2 = model_openai.with_structured_output(HaRazaoParaFlexitarianismo).with_retry(
#     wait_exponential_jitter=True,
#     stop_after_attempt=5
# )


In [132]:
def print_stuff(idx):
    print(texts[idx])
    #print(zsl_classifier2.invoke(texts[idx]))
    print(zsl_classifier.invoke(texts[idx]))

print_stuff(13)

nos últimos anos reduzi a quantidade de carne que eu como no almoco e quase eliminei na janta com os finais de semana sendo um tanto imprevisiveis no entanto nao tenho pretensões de me tornar vegetariano ou vegano devido ao grande planejamento gasto e sacrificio de prazeres pessoais e sociais que essa mudanca acarreta mas admiro a dedicacao de quem decide se tornar
reason='diminui a frequência de consumo'


In [133]:
print_stuff(14)

porque o único animal que eu como e camarao e culinaria japonesa
reason='exclui alguns animais'


In [134]:
print_stuff(15)

porque tomei consciencia de entender o caminho que existe ate virar meu alimento e a partir disso tento fazer escolhas melhores para minha saude dos animais e do planeta
reason='preocupação com a saúde ou ambiente'


In [135]:
print_stuff(16)

porque ja nao como nenhuma carne vermelha nem de porco ha 8 anos
reason='exclui alguns animais'


In [136]:
print_stuff(17)

porque eu nao como carne vermelha nem frango e nem porco mas como peixe
reason='exclui alguns animais'


In [137]:
results = zsl_classifier.batch(texts, config={"max_concurrency": 100})

In [138]:
pd.Series(results).value_counts()

reason='diminui a frequência de consumo'        282
reason='preocupação com a saúde ou ambiente'    203
reason='exclui alguns animais'                   58
reason='outros'                                  28
reason='não considera carne essencial'           14
reason=''                                         6
Name: count, dtype: int64

In [140]:
new_df = pd.DataFrame([{"text": text, "reason": reason.reason} for text, reason in zip(texts, results)])
new_df

,text,reason
0,porque a carne bovina de certa forma me faz ma...,preocupação com a saúde ou ambiente
1,pois limitei o consumo para nao desenvolver ac...,preocupação com a saúde ou ambiente
2,so como carne quando nao ha alternativa nao ti...,diminui a frequência de consumo
3,ja uns anos desde a adolescencia (nos anos 200...,diminui a frequência de consumo
4,porque ao longo do tempo entendi que a reducao...,diminui a frequência de consumo
...,...,...
586,devido a impacto_ambiental e modo de abate dos...,preocupação com a saúde ou ambiente
587,nao sei se entendi a pergunta decidi quase zer...,preocupação com a saúde ou ambiente
588,nao acho que o brasileiro deva renunciar total...,diminui a frequência de consumo
589,porque eu deliberadamente tenho reduzido meu c...,diminui a frequência de consumo


In [142]:
print(new_df.query('reason == "outros"').text.values)

['por gosto (pouca afinidade com carne vermelha) e por consciencia ambiental (direito dos animais)'
 'porque eu como um pouco de tudo desde leguminosass e verdura a carne'
 'pois nao me obrigo a comer nem carne nem 100% vegetal acabo variando bastante focando no meu bem-estar e vontade do momento'
 'apesar de querer ser vegetariano existe fatores que me fazem nao ser ainda e o principal e a cultura familiar carnivora'
 'porque ainda nao consegui sair das carnes gracas a pressao cultural'
 'gostaria de ser vegetariano/vegano mas a praticidade da oferta de alimentos de origem animal acabam retardando minha decisao'
 'o termo e novo pra mim sigo nessa acredito que no futuro sera necessario'
 'nao me considerava ate conhecer um pouco mais sobre o termo dessa forma fui entendendo principalmente com a ajuda da pesquisadora da presente dissertacao nas aulas que tivemos'
 'o fator preco e atenuante, mas sempre gostei de verduras e legumes no geral e tentei inicialmente na pandemia testar se co

In [143]:
print(new_df.query('reason == ""').text.values)

['eu nao sabia que era uma flexitariana ate participar desta pesquisa'
 'eu nao me considero flexitariana, nao ligo que esse seja o jeito que voces chamam a forma como eu como eu me considero onivora apenas'
 'nao conhecia esse termo antes da pesquisa, descobri que sou flexitariano agora'
 'pôde-se dizer que sim, apesar de nao utilizar nunca esse termo do dia a dia'
 'nao tenho uma resposta muito clara para isso nunca antes escutei o termo nem pensei no assunto na verdade nem gosto da palavra ou do conceito por enquanto me parece uma classificacao arbitraria, mas vejamos'
 'nao me considero, nem havia pensado sobre o significado do termo']
